In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from keras.models import load_model
from keras.layers import Lambda

In [2]:
# --- Helper Function to Convert BGR -> RGB for Display ---
def bgr_to_rgb(image):
    """Convert a BGR image (H,W,3) to RGB for display purposes."""
    return image[..., ::-1]

# --- Data Loading and Preprocessing ---
def load_test_data(lr_path, hr_path, n=500):
    lr_list = os.listdir(lr_path)[:n]
    hr_list = os.listdir(hr_path)[:n]

    lr_images = []
    hr_images = []

    # Keep BGR for VGG
    for img in lr_list:
        img_path = os.path.join(lr_path, img)
        img_lr = cv2.imread(img_path)  # BGR
        lr_images.append(img_lr)

    for img in hr_list:
        img_path = os.path.join(hr_path, img)
        img_hr = cv2.imread(img_path)  # BGR
        hr_images.append(img_hr)

    # Optionally filter out low-variance images
    lr_images = [img for img in lr_images if np.std(img) > 0.05]
    hr_images = [img for img in hr_images if np.std(img) > 0.05]

    # Convert to float32, normalize [0,1]
    lr_images = np.array(lr_images, dtype=np.float32) / 255.0
    hr_images = np.array(hr_images, dtype=np.float32) / 255.0

    print(f"LR shape: {lr_images[0].shape}, HR shape: {hr_images[0].shape}")
    print(f"LR range: [{lr_images.min()}, {lr_images.max()}]")
    print(f"HR range: [{hr_images.min()}, {hr_images.max()}]")

    return lr_images, hr_images

In [7]:
# Evaluation Metrics
def evaluate_gan(generator, lr_test, hr_test, batch_size=8):
    psnr_values = []
    ssim_values = []
    mse_values = []
    lr_samples = []
    hr_samples = []
    sr_samples = []
    
    for i in tqdm(range(0, len(lr_test), batch_size), desc="Generating SR images"):
        lr_batch = lr_test[i:i+batch_size]
        hr_batch = hr_test[i:i+batch_size]
        
        sr_batch = generator.predict(lr_batch, verbose=0)
        sr_batch = np.clip(sr_batch, 0.0, 1.0)
        
        for j in range(len(hr_batch)):
            psnr_val = psnr(hr_batch[j], sr_batch[j], data_range=1.0)
            ssim_val = ssim(hr_batch[j], sr_batch[j], data_range=1.0, channel_axis=-1)
            mse_val = np.mean((hr_batch[j] - sr_batch[j]) ** 2)
            
            psnr_values.append(psnr_val)
            ssim_values.append(ssim_val)
            mse_values.append(mse_val)
        
        lr_samples.extend(lr_batch)
        hr_samples.extend(hr_batch)
        sr_samples.extend(sr_batch)
    
    return {
        'psnr': psnr_values,
        'ssim': ssim_values,
        'mse': mse_values,
        'lr_samples': np.array(lr_samples),
        'hr_samples': np.array(hr_samples),
        'sr_samples': np.array(sr_samples)
    }

# Load the trained ESRGAN model with custom objects
generator = load_model('/kaggle/input/enhanced-output/tensorflow2/default/1/esrgan_generator_final.h5', custom_objects={'tf': tf})


In [8]:
# Load test data
lr_path = "/kaggle/input/test-sr/testing dataset/LR"
hr_path = "/kaggle/input/test-sr/testing dataset/HR"
lr_test, hr_test = load_test_data(lr_path, hr_path)

LR shape: (64, 64, 3), HR shape: (256, 256, 3)
LR range: [0.0, 1.0]
HR range: [0.0, 1.0]


In [9]:
# Evaluate the GAN model
gan_metrics = evaluate_gan(generator, lr_test, hr_test)
print("Mean PSNR:", np.mean(gan_metrics['psnr']))
print("Mean SSIM:", np.mean(gan_metrics['ssim']))
print("Mean MSE: ", np.mean(gan_metrics['mse']))

Generating SR images:   0%|          | 0/4 [00:01<?, ?it/s]


NameError: Exception encountered when calling Lambda.call().

[1mname 'tf' is not defined[0m

Arguments received by Lambda.call():
  • inputs=tf.Tensor(shape=(8, 64, 64, 256), dtype=float32)
  • mask=None
  • training=False

In [ ]:
# Sample Comparisons
def plot_gan_samples(gan_metrics, num_samples=30):
    indices = np.random.choice(len(gan_metrics['lr_samples']), num_samples)
    for idx in indices:
        plt.figure(figsize=(12,6))
        # LR Input (BGR -> RGB for display)
        plt.subplot(241)
        lr_bgr = gan_metrics['lr_samples'][idx]
        plt.imshow(bgr_to_rgb(lr_bgr))
        plt.title('LR Input')
        plt.axis('off')
        
        # HR Target (BGR -> RGB for display)
        plt.subplot(242)
        hr_bgr = gan_metrics['hr_samples'][idx]
        plt.imshow(bgr_to_rgb(hr_bgr))
        plt.title('HR Target')
        plt.axis('off')
        
        # SR Output (BGR -> RGB for display)
        plt.subplot(243)
        sr_bgr = gan_metrics['sr_samples'][idx]
        metrics_text = f"PSNR: {gan_metrics['psnr'][idx]:.2f}\nSSIM: {gan_metrics['ssim'][idx]:.4f}\nMSE: {gan_metrics['mse'][idx]:.5f}"
        plt.imshow(bgr_to_rgb(sr_bgr))
        plt.title('ESRGAN Output\n' + metrics_text)
        plt.axis('off')
        
        # Error Map
        plt.subplot(244)
        error = np.abs(hr_bgr - sr_bgr)
        plt.imshow(np.mean(error, axis=-1), cmap='inferno', vmin=0, vmax=0.07)
        plt.title('Pixel Error Heatmap')
        plt.colorbar()
        plt.axis('off')
        plt.tight_layout()
        plt.show()

plot_gan_samples(gan_metrics)

In [ ]:
# Metric Distributions
def plot_gan_metrics(gan_metrics):
    fig, axs = plt.subplots(1, 3, figsize=(18,5))
    metrics = ['psnr', 'ssim', 'mse']
    titles = ['PSNR Distribution', 'SSIM Distribution', 'MSE Distribution']
    colors = ['purple', 'green', 'red']
    for ax, metric, title, color in zip(axs, metrics, titles, colors):
        ax.hist(gan_metrics[metric], bins=30, alpha=0.7, color=color)
        ax.set_title(title)
        ax.set_xlabel(metric.upper())
        ax.set_ylabel('Frequency')
        mean_val = np.mean(gan_metrics[metric])
        ax.axvline(mean_val, color='black', linestyle='dashed', linewidth=2, label=f'Mean: {mean_val:.2f}')
        ax.legend()
    plt.tight_layout()
    plt.show()

plot_gan_metrics(gan_metrics)

In [ ]:
# Error Analysis
def plot_gan_error_analysis(gan_metrics):
    fig, axs = plt.subplots(1, 2, figsize=(12,5))
    errors = np.concatenate([np.abs(hr - sr) for hr, sr in zip(gan_metrics['hr_samples'], gan_metrics['sr_samples'])])
    axs[0].hist(errors.flatten(), bins=50, color='darkorange', density=True)
    axs[0].set_title('Pixel Error Distribution')
    axs[0].set_xlabel('Absolute Error')
    axs[0].set_ylabel('Density')
    
    axs[1].scatter(gan_metrics['psnr'], gan_metrics['ssim'], alpha=0.5)
    axs[1].set_title('PSNR vs SSIM Correlation')
    axs[1].set_xlabel('PSNR')
    axs[1].set_ylabel('SSIM')
    
    psnr_data = np.array(gan_metrics['psnr'])
    ssim_data = np.array(gan_metrics['ssim'])
    X = psnr_data.reshape(-1, 1)
    y = ssim_data
    m, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    
    x_line = np.linspace(0, psnr_data.max(), 100)
    y_line = m[0] * x_line
    axs[1].plot(x_line, y_line, color='red', linestyle='--', label=f'Trend: y = {m[0]:.5f}x')
    axs[1].legend()
    plt.tight_layout()
    plt.show()

plot_gan_error_analysis(gan_metrics)